In [0]:
!pip install lightfm

     |████████████████████████████████| 307kB 2.7MB/s 
  Created wheel for lightfm: filename=lightfm-1.15-cp36-cp36m-linux_x86_64.whl size=707612 sha256=13ca9c540d2926f37087dc40a727ac1bf1ccc1b6f178c96c84586e7a6ea00ae4
  Stored in directory: /root/.cache/pip/wheels/eb/bb/ac/188385a5da6627956be5d9663928483b36da576149ab5b8f79
Successfully built lightfm


In [0]:
import pandas as pd # pandas for data manipulation
import numpy as np # numpy for sure
import numpy.matlib
import matplotlib.pyplot as plt
from lightfm import LightFM # model
import scipy
import pickle

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
#import saved lightFM model and user-production matrix
path = '/content/drive/My Drive/Insight/'
filename = path+'model_lightfm_v2.pkl'
model = pickle.load(open(filename, 'rb'))

user_prod_interaction = scipy.sparse.load_npz(path+'user_prod_interaction.npz')

In [0]:
#import list of users
df_users = pd.read_csv(path+'users_order_by_prev_dept.csv')
df_users.drop(['Unnamed: 0'],axis=1,inplace=True)
display(df_users)

,order_id,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order,user_id_order,user_id_prevorder,order_id_prev,organic,any_organic,"('department', 'alcohol')","('department', 'babies')","('department', 'bakery')","('department', 'beverages')","('department', 'breakfast')","('department', 'bulk')","('department', 'canned goods')","('department', 'dairy eggs')","('department', 'deli')","('department', 'dry goods pasta')","('department', 'frozen')","('department', 'household')","('department', 'international')","('department', 'meat seafood')","('department', 'missing')","('department', 'other')","('department', 'pantry')","('department', 'personal care')","('department', 'pets')","('department', 'produce')","('department', 'snacks')"
0,1187899,1,train,11,4,8,14.0,1.011,1.010,2550362.0,3.0,True,0,0,0,2,1,0,0,3,0,0,0,0,0,0,0,0,0,0,0,0,3
1,1492625,2,train,15,1,11,30.0,2.015,2.014,839880.0,6.0,True,0,0,0,2,0,0,0,6,0,0,1,0,1,0,0,0,1,0,0,4,1
2,2196797,5,train,5,0,11,6.0,5.005,5.004,157374.0,5.0,True,0,0,0,0,0,0,1,4,1,0,0,0,1,0,0,0,0,0,0,5,0
3,525192,7,train,21,2,11,6.0,7.021,7.020,2452257.0,1.0,True,0,0,0,3,0,0,1,2,0,0,0,0,0,1,0,0,0,0,0,4,1
4,880375,8,train,4,1,14,10.0,8.004,8.003,2570360.0,9.0,True,0,0,0,0,0,0,2,3,0,0,0,0,0,0,0,0,1,0,0,7,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131204,2585586,206199,train,20,2,16,30.0,206199.020,206199.019,1932632.0,1.0,True,0,0,1,2,0,0,0,5,0,1,2,4,0,0,2,0,1,0,0,1,9
131205,943915,206200,train,24,6,19,6.0,206200.024,206200.023,1909878.0,10.0,True,0,0,0,1,1,0,1,2,1,0,0,0,0,1,0,0,6,0,0,16,1
131206,2371631,206203,train,6,4,19,30.0,206203.006,206203.005,133582.0,4.0,True,0,0,0,3,0,0,2,1,0,0,4,1,0,0,0,0,1,0,0,1,7
131207,1716008,206205,train,4,1,16,10.0,206205.004,206205.003,414137.0,8.0,True,0,0,1,1,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,3,0


In [0]:
#import products table and top 200 products table
products = pd.read_csv(path+'products.csv')
top200products = pd.read_csv(path+'top200_products.csv')
top200products.drop(['Unnamed: 0'],axis=1,inplace=True)

#rank top products by frequency
top200products['freq_rank'] = np.arange(1,201,1)

In [0]:
display(top200products.head(5))
top200products['product_id'].head(5).values

,product_id,frequency,product_name,organic,freq_rank
0,24852,18726,Banana,0,1
1,13176,15480,Bag of Organic Bananas,1,2
2,21137,10894,Organic Strawberries,1,3
3,21903,9784,Organic Baby Spinach,1,4
4,47626,8135,Large Lemon,0,5


array([24852, 13176, 21137, 21903, 47626])

In [0]:
len(df_users['user_id'].values[0:50000])

50000

In [0]:
%%time

num_top_prods_to_rec = 100
recom_products_master = pd.DataFrame(None,columns=
  ['product_id','product_name','organic','freq_rank','user_id','rec_rank'])

#Loop through all users and get recommendations:
# if more time, can do this through passing in the right-sized np arrays into
#  lightfm.predict()
for user_id_to_get_rec in df_users['user_id'].values[50000:]:
  # print(user_id_to_get_rec)

  user_id_single = [user_id_to_get_rec]
  row_ixs = [user_id-1 for user_id in user_id_single] #subtract one to get indices
  # row_ixs = np.array(row_ixs)

  # # Look for this user in the user_prod_interaction matrix
  # col_ixs = user_prod_interaction.tocsr()[row_ixs].indices 
  # print(col_ixs) #these are the column indices in the matrix. 

  # # Need to add 1 to get product ids.
  # user_prod_ids = col_ixs+1

  # # This user purchased these items:
  # purch_products = products.loc[products['product_id'].isin(user_prod_ids)]
  # print('Purchased products:')
  # display(purch_products)

  # Scores from LightFM model prediction for specific products:
  products_for_rec = top200products.head(num_top_prods_to_rec)
  # prod_ixs = np.arange(user_prod_interaction.shape[1]) #all
  prod_ixs = products_for_rec['product_id'].values - 1
  #remember prod_ixs are one off the actual prod_ids
  scores = model.predict(user_ids = row_ixs, item_ids = prod_ixs) 

  # print(prod_ixs)
  # print(scores)
  # print(np.argsort(-scores))
  # print(prod_ixs[np.argsort(-scores)])

  # get the re-ordered product list
  recom_products = products_for_rec.loc[np.argsort(-scores)]
  recom_products['user_id'] = user_id_to_get_rec 
  recom_products['rec_rank'] = np.arange(1,num_top_prods_to_rec+1,1) 
  recom_products.drop('frequency',axis=1,inplace=True)

  # print('Recommended products:')
  # display(recom_products)
  # display(recom_products.head(20))
  recom_products_master = recom_products_master.append(recom_products.head(20))
display(recom_products_master.sample())
recom_products_master.to_csv('top20_products_recom.csv')

,product_id,product_name,organic,freq_rank,user_id,rec_rank
4,47626,Large Lemon,0,5,190767,4


CPU times: user 3h 2min 10s, sys: 1min 28s, total: 3h 3min 39s
Wall time: 3h 3min 47s


In [0]:
len(recom_products_master)

1000000